In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, BoundaryNorm
import imageio.v2 as imageio
from io import BytesIO
from tqdm import tqdm
import geopandas as gpd
from pykrige.ok import OrdinaryKriging
import os

# ==============================
# User settings
# ==============================
CSV_FILE   = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_corrected.csv"
PASIG_SHP  = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR    = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\GIFs"
os.makedirs(OUT_DIR, exist_ok=True)

qa_mode   = False          # True = show first 5 frames only, False = render GIF
months    = [5,6]           # choose which months to run
frame_dur = 1.0           # seconds per frame in GIF
fig_size  = (5, 5)        # fixed output size (inches)

# AQI bins & colors (US EPA)
aqi_bins   = [0, 50, 100, 150, 200, 300, 500]
aqi_colors = ["#00E400", "#FFFF00", "#FF7E00",
              "#FF0000", "#8F3F97", "#7E0023"]
aqi_labels = [
    "Good (0-50)",
    "Moderate (51-100)",
    "Unhealthy for Sensitive (101-150)",
    "Unhealthy (151-200)",
    "Very Unhealthy (201-300)",
    "Hazardous (301-500)"
]
cmap = ListedColormap(aqi_colors)
norm = BoundaryNorm(aqi_bins, cmap.N)

# ==============================
# Functions
# ==============================
def pm25_to_aqi(pm25):
    """Convert PM2.5 to AQI using US EPA breakpoints."""
    breakpoints = [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.4, 301, 500)
    ]
    for (c_low, c_high, aqi_low, aqi_high) in breakpoints:
        if c_low <= pm25 <= c_high:
            return ((aqi_high - aqi_low) / (c_high - c_low)) * (pm25 - c_low) + aqi_low
    return 500

# ==============================
# Load data
# ==============================
df = pd.read_csv(CSV_FILE)
df["datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"])
df = df.set_index("datetime").sort_index()

# Convert PM2.5 → AQI
df["AQI"] = df["pm25"].apply(pm25_to_aqi)

# load Pasig shapefile
pasig = gpd.read_file(PASIG_SHP)
pasig_bounds = pasig.total_bounds

# ==============================
# Loop over months
# ==============================
for month in months:
    group = df[(df.index.year == 2025) & (df.index.month == month)]
    if group.empty:
        print(f"⚠️ No data found for 2025-{month:02d}")
        continue

    if qa_mode:
        print(f"📅 QA Preview for 2025-{month:02d} (first 5 frames)...")
        iter_group = group.groupby(group.index)
    else:
        out_gif = os.path.join(OUT_DIR, f"2025_{month:02d}_AQI.gif")
        writer = imageio.get_writer(out_gif, mode="I", duration=frame_dur)
        iter_group = tqdm(group.groupby(group.index), desc=f"Rendering 2025-{month:02d}")

    for i, (t, g) in enumerate(iter_group):
        if g.shape[0] <= 3:  # only run kriging if >3 stations
            continue
        if qa_mode and i >= 5:   # preview first 5 frames only
            break

        # Extract station data
        lons, lats, values = g["longitude"].values, g["latitude"].values, g["AQI"].values

        # Grid for interpolation
        grid_lon = np.linspace(pasig_bounds[0]-0.005, pasig_bounds[2]+0.005, 100)
        grid_lat = np.linspace(pasig_bounds[1]-0.005, pasig_bounds[3]+0.005, 100)

        # Ordinary Kriging
        try:
            OK = OrdinaryKriging(lons, lats, values, variogram_model="linear",
                                 verbose=False, enable_plotting=False)
            z, ss = OK.execute("grid", grid_lon, grid_lat)
        except Exception as e:
            print(f"⚠️ Kriging failed at {t}: {e}")
            continue

        # Plot
        fig, ax = plt.subplots(figsize=fig_size)

        # Pasig boundary
        pasig.boundary.plot(ax=ax, edgecolor="black", linewidth=0.3, zorder=1)

        # Interpolated AQI surface (discrete bins)
        im = ax.imshow(z, extent=(grid_lon.min(), grid_lon.max(),
                                  grid_lat.min(), grid_lat.max()),
                       cmap=cmap, norm=norm, alpha=0.6, origin="lower", zorder=0)

        # Plot stations
        ax.scatter(lons, lats, c=[cmap(norm(v)) for v in values],
                   s=60, edgecolor="k", linewidth=0.5, zorder=2)
        for x, y, val in zip(lons, lats, values):
            ax.text(x+0.001, y+0.001, f"{int(val)}", fontsize=6, zorder=3)

        ax.set_title(f"PM2.5 AQI Kriging - {t.strftime('%Y-%m-%d %H:%M')}", fontsize=10)
        ax.set_xlim(pasig_bounds[0]-0.01, pasig_bounds[2]+0.01)
        ax.set_ylim(pasig_bounds[1]-0.01, pasig_bounds[3]+0.01)

        # Custom AQI legend
        patches = [mpatches.Patch(color=aqi_colors[j], label=aqi_labels[j])
                   for j in range(len(aqi_labels))]
        ax.legend(handles=patches, loc="lower left", fontsize=6, frameon=True)

        if qa_mode:
            plt.show()
        else:
            buf = BytesIO()
            plt.savefig(buf, format="png", dpi=100, bbox_inches="tight")
            buf.seek(0)
            writer.append_data(imageio.imread(buf))
        plt.close(fig)

    if not qa_mode:
        writer.close()
        print(f"✅ Saved {out_gif}")

Rendering 2025-05:   1%|          | 4/739 [00:01<04:01,  3.04it/s]

⚠️ Kriging failed at 2025-05-01 04:00:00: singular matrix


Rendering 2025-05:  10%|█         | 75/739 [00:26<03:16,  3.39it/s]

⚠️ Kriging failed at 2025-05-04 03:00:00: singular matrix


Rendering 2025-05:  34%|███▎      | 249/739 [01:29<02:55,  2.79it/s]

...

⚠️ Kriging failed at 2025-06-15 23:00:00: singular matrix


Rendering 2025-06:  56%|█████▌    | 395/706 [02:08<01:35,  3.25it/s]

⚠️ Kriging failed at 2025-06-18 01:00:00: singular matrix


Rendering 2025-06: 100%|██████████| 706/706 [03:48<00:00,  3.08it/s]


✅ Saved C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\GIFs\2025_06_AQI.gif
